# Visualize VGG16 Filters

Visualization of the filters of VGG16, via gradient ascent in input space.

In [4]:
from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
from keras.applications import vgg16
from keras import backend as K

Using Theano backend.


In [5]:
# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

## Create a function to extract and display the generated input

In [6]:
# util function to convert a tensor into a valid image

def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x



## Build the VGG16 network with ImageNet weights

Note that we only go up to the last convolutional layer --we don't include fully-connected layers. The reason is that adding the fully connected layers forces you to use a fixed input size for the model (224x224, the original ImageNet format). By only keeping the convolutional modules, our model can be adapted to arbitrary input sizes.

In [7]:
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0      

## Define a loss function 

This loss function will seek to maximize the activation of a specific filter (filter_index) in a specific layer (layer_name)

In [8]:
# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

kept_filters = []
for filter_index in range(0, 200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img



Processing filter 0
Current loss value: 12.368642807006836
Current loss value: 23.671627044677734
Current loss value: 42.381229400634766
Current loss value: 65.37151336669922
Current loss value: 85.69359588623047
Current loss value: 99.78841400146484
Current loss value: 127.03534698486328
Current loss value: 149.66452026367188
Current loss value: 169.27783203125
Current loss value: 194.4546661376953
Current loss value: 218.1980438232422
Current loss value: 236.95928955078125
Current loss value: 263.23162841796875
Current loss value: 285.45343017578125
Current loss value: 308.71783447265625
Current loss value: 330.14715576171875
Current loss value: 351.1011047363281
Current loss value: 373.4712219238281
Current loss value: 392.3348083496094
Current loss value: 413.6596374511719
Filter 0 processed in 24s
Processing filter 1
Current loss value: 0.0
Filter 1 processed in 1s
Processing filter 2
Current loss value: 8.815566062927246
Current loss value: 26.908369064331055
Current loss value: 

Current loss value: 0.0
Filter 12 processed in 1s
Processing filter 13
Current loss value: 6.083409309387207
Current loss value: 66.41938018798828
Current loss value: 113.48970794677734
Current loss value: 152.35116577148438
Current loss value: 193.04087829589844
Current loss value: 230.1475830078125
Current loss value: 265.9126892089844
Current loss value: 298.7203369140625
Current loss value: 330.4977722167969
Current loss value: 364.93768310546875
Current loss value: 398.5561828613281
Current loss value: 428.3167419433594
Current loss value: 463.47747802734375
Current loss value: 493.3843688964844
Current loss value: 523.8150634765625
Current loss value: 553.3346557617188
Current loss value: 579.9303588867188
Current loss value: 607.713623046875
Current loss value: 633.1329345703125
Current loss value: 662.248291015625
Filter 13 processed in 8s
Processing filter 14
Current loss value: 4.796126365661621
Current loss value: 35.18335723876953
Current loss value: 98.74899291992188
Curre

Current loss value: 599.0634765625
Current loss value: 669.3225708007812
Current loss value: 752.0789184570312
Current loss value: 826.4656372070312
Current loss value: 903.2753295898438
Current loss value: 980.2494506835938
Current loss value: 1062.7899169921875
Filter 27 processed in 8s
Processing filter 28
Current loss value: 1.194626808166504
Current loss value: 26.927234649658203
Current loss value: 83.96559143066406
Current loss value: 150.6856231689453
Current loss value: 213.692626953125
Current loss value: 262.5455017089844
Current loss value: 309.1175842285156
Current loss value: 361.2740478515625
Current loss value: 403.1582946777344
Current loss value: 455.4643859863281
Current loss value: 494.2698669433594
Current loss value: 536.0096435546875
Current loss value: 567.5340576171875
Current loss value: 606.19873046875
Current loss value: 639.49609375
Current loss value: 674.2373046875
Current loss value: 706.5465698242188
Current loss value: 747.13623046875
Current loss valu

Current loss value: 364.9054870605469
Current loss value: 410.89202880859375
Current loss value: 448.8857421875
Current loss value: 483.3177795410156
Current loss value: 513.9097900390625
Current loss value: 552.392333984375
Filter 39 processed in 8s
Processing filter 40
Current loss value: 0.7768726944923401
Current loss value: 3.23960018157959
Current loss value: 21.15452766418457
Current loss value: 63.554054260253906
Current loss value: 131.28121948242188
Current loss value: 210.23532104492188
Current loss value: 282.446044921875
Current loss value: 361.6301574707031
Current loss value: 457.1392517089844
Current loss value: 560.1746826171875
Current loss value: 648.6728515625
Current loss value: 727.666015625
Current loss value: 799.8228149414062
Current loss value: 889.5393676757812
Current loss value: 956.9638061523438
Current loss value: 1035.6978759765625
Current loss value: 1098.2945556640625
Current loss value: 1169.7720947265625
Current loss value: 1231.3575439453125
Current

Current loss value: 1208.1815185546875
Filter 49 processed in 8s
Processing filter 50
Current loss value: 2.1621181964874268
Current loss value: 4.145421028137207
Current loss value: 10.450810432434082
Current loss value: 19.86762809753418
Current loss value: 33.79091262817383
Current loss value: 48.661624908447266
Current loss value: 63.74822998046875
Current loss value: 88.40054321289062
Current loss value: 109.86241149902344
Current loss value: 132.75241088867188
Current loss value: 158.44705200195312
Current loss value: 186.1185760498047
Current loss value: 218.3110809326172
Current loss value: 246.7108154296875
Current loss value: 273.39501953125
Current loss value: 304.4980163574219
Current loss value: 329.94598388671875
Current loss value: 357.6765441894531
Current loss value: 387.4296569824219
Current loss value: 417.6482238769531
Filter 50 processed in 8s
Processing filter 51
Current loss value: 1.1207586526870728
Current loss value: 20.40886878967285
Current loss value: 84.66

...

INFO (theano.gof.compilelock): Waiting for existing lock by process '72673' (I am process '73077')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/YOUR_USER/.theano/compiledir_Darwin-16.6.0-x86_64-i386-64bit-i386-3.6.1-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '72673' (I am process '73077')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/YOUR_USER/.theano/compiledir_Darwin-16.6.0-x86_64-i386-64bit-i386-3.6.1-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '72673' (I am process '73077')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/YOUR_USER/.theano/compiledir_Darwin-16.6.0-x86_64-i386-64bit-i386-3.6.1-64/lock_dir


Current loss value: 5.318907260894775
Current loss value: 45.00846862792969
Current loss value: 63.11859130859375
Current loss value: 80.04740905761719
Current loss value: 119.55887603759766
Current loss value: 162.90040588378906
Current loss value: 203.64312744140625
Current loss value: 232.85867309570312
Current loss value: 276.15447998046875
Current loss value: 323.4171142578125
Current loss value: 335.1720275878906
Current loss value: 383.4561767578125
Current loss value: 416.9403991699219
Current loss value: 456.112060546875
Current loss value: 498.19970703125
Current loss value: 535.1488647460938
Current loss value: 580.11328125
Current loss value: 614.8665161132812
Current loss value: 633.5018920898438
Current loss value: 678.111328125
Filter 152 processed in 39s
Processing filter 153
Current loss value: 1.3801673650741577
Current loss value: 1.9737216234207153
Current loss value: 10.833495140075684
Current loss value: 41.51381301879883
Current loss value: 90.76924896240234
Curr

Current loss value: 0.5419502854347229
Current loss value: 15.489192962646484
Current loss value: 67.84156036376953
Current loss value: 130.1534423828125
Current loss value: 209.73822021484375
Current loss value: 304.35430908203125
Current loss value: 396.1256408691406
Current loss value: 478.6799621582031
Current loss value: 558.0706787109375
Current loss value: 636.1292724609375
Current loss value: 703.7367553710938
Current loss value: 767.9826049804688
Current loss value: 838.151611328125
Current loss value: 897.678466796875
Current loss value: 971.2354125976562
Current loss value: 1024.80078125
Current loss value: 1085.978515625
Current loss value: 1142.087158203125
Current loss value: 1194.56494140625
Filter 163 processed in 10s
Processing filter 164
Current loss value: 0.0
Filter 164 processed in 2s
Processing filter 165
Current loss value: 2.0405492782592773
Current loss value: 14.655295372009277
Current loss value: 42.995872497558594
Current loss value: 73.4898681640625
Current

Current loss value: 0.5065935850143433
Current loss value: 15.156623840332031
Current loss value: 75.95596313476562
Current loss value: 149.03561401367188
Current loss value: 218.45263671875
Current loss value: 291.33306884765625
Current loss value: 359.11529541015625
Current loss value: 419.9208984375
Current loss value: 480.39947509765625
Current loss value: 539.758056640625
Current loss value: 595.8117065429688
Current loss value: 648.23779296875
Current loss value: 700.1998291015625
Current loss value: 754.31396484375
Current loss value: 803.4493408203125
Current loss value: 855.2537841796875
Current loss value: 914.724365234375
Current loss value: 968.6709594726562
Current loss value: 1025.0360107421875
Current loss value: 1078.3536376953125
Filter 176 processed in 10s
Processing filter 177
Current loss value: 7.494861125946045
Current loss value: 18.816179275512695
Current loss value: 39.842098236083984
Current loss value: 67.44925689697266
Current loss value: 111.32730102539062


Current loss value: 909.0684814453125
Filter 188 processed in 7s
Processing filter 189
Current loss value: 0.5670349597930908
Current loss value: 3.3536319732666016
Current loss value: 20.231042861938477
Current loss value: 60.65279769897461
Current loss value: 106.93583679199219
Current loss value: 160.92556762695312
Current loss value: 195.47679138183594
Current loss value: 237.55352783203125
Current loss value: 274.5997009277344
Current loss value: 322.7433166503906
Current loss value: 357.6360778808594
Current loss value: 393.8175048828125
Current loss value: 423.0922546386719
Current loss value: 460.97802734375
Current loss value: 493.55938720703125
Current loss value: 534.054443359375
Current loss value: 566.150390625
Current loss value: 599.3922729492188
Current loss value: 629.29638671875
Current loss value: 662.1373901367188
Filter 189 processed in 8s
Processing filter 190
Current loss value: 3.4644651412963867
Current loss value: 16.373262405395508
Current loss value: 63.7076

In [9]:
# save the result to disk
imsave('img/stitched_filters.png' , stitched_filters)

## View the Image of VGG16 Filters

![mnist_digits](img/stitched_filters.png "Title")

More Details on [How Convolutional Neural Networks see the world](https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html)